<a href="https://colab.research.google.com/github/alxmarqs/LLMtopics/blob/main/1_1_tokenizacao_pratica_adaptjson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aula 2 - Tokenização

Corpus utilizado: **100 frases de Marco Aurélio** (*Meditações*).

Estrutura desta aula:
1. Pré-tokenização
2. Treinamento (BPE e WordPiece)
3. Pipeline de Encode completo
4. ByteLevel vs SentencePiece
5. Avaliação comparativa

## Configuração: Corpus de Marco Aurélio

In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tokenizers import Tokenizer, models, trainers, pre_tokenizers, decoders

# ------------------------------------------------------------------------------
# Corpus: 100 frases de Marco Aurélio
# ------------------------------------------------------------------------------
frases_marco_aurelio = [
    "A felicidade da sua vida depende da qualidade dos seus pensamentos.",
    "Não perca mais tempo discutindo sobre o que um bom homem deve ser. Seja um.",
    "Você tem poder sobre sua mente, não sobre eventos externos. Perceba isso e encontrará a força.",
    "Tudo o que ouvimos é uma opinião, não um fato.",
    "Tudo o que vemos é uma perspectiva, não a verdade.",
    "A alma é tingida pela cor dos seus pensamentos.",
    "A melhor vingança é ser diferente de quem causou o dano.",
    "Aceite as coisas às quais o destino o ata e ame as pessoas com quem o destino o une.",
    "Faça cada coisa na vida como se fosse a última.",
    "A morte sorri para todos nós; tudo o que um homem pode fazer é sorrir de volta.",
    "O que não é bom para a colmeia também não é bom para a abelha.",
    "A perda nada mais é do que mudança, e a mudança é o deleite da natureza.",
    "Se algo é possível para qualquer homem, considere que também é possível para você.",
    "A vida de um homem é o que seus pensamentos fazem dela.",
    "Olhe para dentro. Dentro está a fonte do bem, e ela sempre jorrará se você sempre cavar.",
    "Aquele que vive em harmonia consigo mesmo vive em harmonia com o universo.",
    "Muitas vezes me perguntei como é que todo homem se ama mais do que a todos os outros homens.",
    "A arte de viver assemelha-se mais à luta do que à dança.",
    "O universo é mudança; a vida é opinião.",
    "Não aja como se fosse viver dez mil anos. A morte paira sobre você.",
    "Enquanto viver e enquanto puder, seja bom.",
    "Quão ridículo e quão estranho é surpreender-se com qualquer coisa que acontece na vida.",
    "O objetivo da vida não é estar do lado da maioria, mas escapar de se encontrar nas fileiras dos loucos.",
    "O homem deve ter medo não da morte, mas de nunca começar a viver.",
    "A pobreza é a mãe do crime.",
    "Lembre-se de que tudo o que acontece, acontece como deveria.",
    "Nada acontece a homem algum que ele não seja formado pela natureza para suportar.",
    "A rejeição é a sua própria defesa.",
    "A primeira regra é manter o espírito tranquilo.",
    "A segunda regra é olhar as coisas de frente e saber o que elas são.",
    "Não se deixe levar pelo futuro.",
    "Sempre que estiver prestes a apontar um defeito em outra pessoa, questione seus próprios defeitos.",
    "A vida não é boa nem má, mas apenas o lugar para o bem e o mal.",
    "Se não for certo, não faça; se não for verdade, não diga.",
    "Concentre-se a cada minuto como um romano e um homem.",
    "Faça o que tem à sua frente com seriedade precisa e genuína.",
    "Liberte-se de todas as outras distrações.",
    "A brevidade da vida é a única certeza.",
    "Em breve você terá esquecido tudo; em breve todos terão esquecido você.",
    "O tempo é um rio, um fluxo violento de eventos.",
    "Assim que algo é avistado, é arrastado e outra coisa toma o seu lugar.",
    "Pense na beleza da vida. Observe as estrelas e veja-se correndo com elas.",
    "Tudo o que é belo em si mesmo é completo em si mesmo e não precisa de elogios.",
    "O elogio não torna nada melhor ou pior.",
    "Se você está angustiado por qualquer coisa externa, a dor não se deve à coisa em si.",
    "Você tem o poder de revogar sua estimativa a qualquer momento.",
    "Seja como o promontório contra o qual as ondas quebram continuamente.",
    "Permaneça firme e dome a fúria da água ao seu redor.",
    "Não fui ferido pelo presente nem tenho medo do futuro.",
    "Adapte-se ao ambiente em que sua sorte o colocou.",
    "Ame verdadeiramente os companheiros com quem o destino o ordenou caminhar.",
    "Olhe para o passado, para os impérios em ascensão e queda.",
    "Não há nada de novo sob o sol.",
    "Nenhuma pessoa tem o poder de ter tudo o que deseja.",
    "Está em seu poder não querer o que você não tem.",
    "A tranquilidade não é nada além da boa ordenação da mente.",
    "O mundo é uma cidade.",
    "Para viver feliz, basta muito pouco.",
    "Tudo está interligado como uma teia sagrada.",
    "Nenhuma coisa é estranha à outra.",
    "Os homens existem para o bem uns dos outros.",
    "Ensine-os ou suporte-os.",
    "A injustiça é uma impiedade.",
    "Aquele que comete injustiça, comete injustiça contra si mesmo.",
    "Muitas vezes peca quem deixa de fazer algo, não apenas quem faz algo.",
    "Comece cada dia dizendo a si mesmo: Hoje encontrarei interferência, ingratidão, insolência.",
    "Mas eu vi a beleza do bem e a feiura do mal.",
    "Reconheço que o malfeitor tem uma natureza semelhante à minha.",
    "Ninguém pode me prejudicar, pois ninguém pode me envolver em baixeza.",
    "Nascemos para trabalhar juntos como pés, mãos e olhos.",
    "Obstruir uns aos outros é antinatural.",
    "Sentir raiva e virar as costas para alguém é obstrução.",
    "O que morre não cai fora do mundo.",
    "Se fica aqui, muda aqui e se dissolve em suas partes apropriadas.",
    "A mudança não é um mal, assim como a persistência no novo estado não é um bem.",
    "Lembre-se de quanto tempo você adiou essas coisas.",
    "Quantas vezes você recebeu uma oportunidade dos deuses e não a usou.",
    "Você deve finalmente perceber de que tipo de mundo você é uma parte.",
    "Existe um limite para o seu tempo.",
    "O tempo nunca retornará.",
    "Descarte sua sede de livros, para que não morra murmurando.",
    "Morra verdadeiramente resignado e grato de coração aos deuses.",
    "Considere continuamente como todas as coisas que existem agora existiram no passado.",
    "Dramas e cenas inteiras são sempre os mesmos, apenas os atores mudam.",
    "Não despreze a morte, mas aceite-a de bom grado.",
    "A morte é uma das coisas que a natureza deseja.",
    "Busque a sabedoria, não o conhecimento.",
    "O conhecimento é o acúmulo de fatos, a sabedoria é a simplificação.",
    "A melhor maneira de se vingar de um inimigo é não se assemelhar a ele.",
    "Onde um homem pode viver, ele também pode viver bem.",
    "É possível viver bem em um palácio? Sim.",
    "Então é possível viver bem em qualquer lugar.",
    "Olhe sob a superfície; não deixe que a qualidade ou o valor de nada lhe escape.",
    "A felicidade é um bom fluxo da vida.",
    "Temos dois ouvidos e uma boca, para ouvirmos mais do que falamos.",
    "O homem conquista o mundo conquistando a si mesmo.",
    "Nenhuma perda deve ser mais dolorosa para nós do que a perda do tempo.",
    "Siga para onde a razão levar.",
    "O destino é a cadeia infinita de causalidade, por onde as coisas existem.",
]

# Salva como JSONL (formato recomendado pela biblioteca tokenizers)
ARQUIVO_CORPUS = "corpus_marco_aurelio.jsonl"
with open(ARQUIVO_CORPUS, "w", encoding="utf-8") as f:
    for frase in frases_marco_aurelio:
        f.write(json.dumps({"text": frase}, ensure_ascii=False) + "\n")

def carregar_corpus_jsonl(caminho):
    with open(caminho, "r", encoding="utf-8") as f:
        for line in f:
            yield json.loads(line)["text"]

print(f"Corpus criado: {len(frases_marco_aurelio)} frases.")
print(f"Exemplo: {frases_marco_aurelio[0]}")

---
## Parte 1 - Pré-tokenização

Comparamos quatro estratégias de pré-tokenização sobre uma frase do corpus.

In [ ]:
from tokenizers import Tokenizer, models, pre_tokenizers

frase = "A felicidade da sua vida depende da qualidade dos seus pensamentos."

# --- Whitespace ---
tok_ws = Tokenizer(models.BPE())
tok_ws.pre_tokenizer = pre_tokenizers.Whitespace()
print("Whitespace:")
print(tok_ws.pre_tokenizer.pre_tokenize_str(frase))

### Punctuation + Whitespace

In [ ]:
tok_punc = Tokenizer(models.BPE())
tok_punc.pre_tokenizer = pre_tokenizers.Sequence([
    pre_tokenizers.Whitespace(),
    pre_tokenizers.Punctuation()
])
print("Punctuation + Whitespace:")
print(tok_punc.pre_tokenizer.pre_tokenize_str(frase))

### ByteLevel (estilo GPT-2)

In [ ]:
tok_byte = Tokenizer(models.BPE())
tok_byte.pre_tokenizer = pre_tokenizers.ByteLevel()
print("ByteLevel (GPT-2 style):")
print(tok_byte.pre_tokenizer.pre_tokenize_str(frase))

### Metaspace (SentencePiece style)

In [ ]:
tok_meta = Tokenizer(models.BPE())
tok_meta.pre_tokenizer = pre_tokenizers.Metaspace()
print("Metaspace (SentencePiece style):")
print(tok_meta.pre_tokenizer.pre_tokenize_str(frase))

---
## Parte 2 - Treinamento

Treinamos dois modelos sobre o corpus de Marco Aurélio:
- **BPE** (Byte Pair Encoding)
- **WordPiece** (estilo BERT)

In [ ]:
from tokenizers import trainers

# Visão geral dos algoritmos
print("=== BPE ===")
print("1. Começa com todos os caracteres do corpus como tokens.")
print("2. Encontra e une o par de tokens mais frequente em um novo token.")
print("3. Repete até atingir o tamanho de vocabulário desejado.\n")

print("=== WordPiece ===")
print("1. Similar ao BPE, mas usa uma função de pontuação: freq(AB) / (freq(A) * freq(B)).")
print("2. Favorece pares que aumentam a verossimilhança do modelo.")
print("3. Sufixos são marcados com '##' para indicar continuidade.\n")

VOCAB_SIZE    = 1000
SPECIAL_TOKENS = ["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"]

# --- Treino BPE ---
print("Treinando BPE...")
tokenizer_bpe = Tokenizer(models.BPE(unk_token="[UNK]"))
tokenizer_bpe.pre_tokenizer = pre_tokenizers.Whitespace()
trainer_bpe = trainers.BpeTrainer(vocab_size=VOCAB_SIZE, special_tokens=SPECIAL_TOKENS)
tokenizer_bpe.train_from_iterator(carregar_corpus_jsonl(ARQUIVO_CORPUS), trainer=trainer_bpe)
tokenizer_bpe.save("tokenizer_bpe.json")

# --- Treino WordPiece ---
print("Treinando WordPiece...")
tokenizer_wp = Tokenizer(models.WordPiece(unk_token="[UNK]"))
tokenizer_wp.pre_tokenizer = pre_tokenizers.Whitespace()
tokenizer_wp.decoder = decoders.WordPiece()
trainer_wp = trainers.WordPieceTrainer(vocab_size=VOCAB_SIZE, special_tokens=SPECIAL_TOKENS)
tokenizer_wp.train_from_iterator(carregar_corpus_jsonl(ARQUIVO_CORPUS), trainer=trainer_wp)
tokenizer_wp.save("tokenizer_wp.json")

# Exibe parte do vocabulário BPE
vocab = tokenizer_bpe.get_vocab()
print(f"\nTamanho do vocabulário BPE: {len(vocab)}")
sorted_vocab = sorted(vocab.items(), key=lambda kv: kv[1])[:20]
for token, idx in sorted_vocab:
    print(f"{idx:>3} → {repr(token)}")

### Teste rápido dos modelos treinados

In [ ]:
tok_bpe = Tokenizer.from_file("tokenizer_bpe.json")
tok_wp  = Tokenizer.from_file("tokenizer_wp.json")

textos_teste = [
    "A vida de um homem é o que seus pensamentos fazem dela.",
    "Anticonstitucionalissimamente",
    "Data science e machine learning transformam o agro."
]

print("\nTokenização de exemplos (BPE):")
for t in textos_teste:
    out = tok_bpe.encode(t)
    print(f"  Texto  : {t}")
    print(f"  Tokens : {out.tokens}")
    print(f"  IDs    : {out.ids}\n")

---
## Parte 3 - Encode

Pipeline completo: **normalização → pré-tokenização → modelo → pós-processamento**

In [ ]:
from tokenizers import normalizers, processors
from tokenizers.normalizers import NFD, StripAccents, Lowercase
from tokenizers.pre_tokenizers import Whitespace, Digits, Sequence
from tokenizers.processors import TemplateProcessing

print("### Pipeline de tokenização ###")
print(" 1. Normalization")
print(" 2. Pre-tokenization")
print(" 3. Model")
print(" 4. Post-processing\n")

# Carrega o tokenizador BPE treinado no corpus de Marco Aurélio
tokenizer = Tokenizer.from_file("tokenizer_bpe.json")

# ------------------------------------------------------------------
# 1. Normalization
# ------------------------------------------------------------------
print("# Normalization")
normalizer = normalizers.Sequence([
    NFD(),          # decompõe acentos em caracteres separados
    Lowercase(),    # tudo minúsculo
    StripAccents()  # remove acentos
])
texto_teste = "A Tranqüilidade da Mente é Essencial."
print("Antes :", texto_teste)
print("Depois:", normalizer.normalize_str(texto_teste), "\n")
tokenizer.normalizer = normalizer

# ------------------------------------------------------------------
# 2. Pre-tokenization
# ------------------------------------------------------------------
print("# Pre-tokenization")
pre_tok = Sequence([
    Whitespace(),
    Digits(individual_digits=True)
])
texto_numeros = "Marco Aurélio nasceu em 121 d.C. e morreu em 180 d.C."
print("Pré-tokenização:", pre_tok.pre_tokenize_str(texto_numeros), "\n")
tokenizer.pre_tokenizer = pre_tok

# ------------------------------------------------------------------
# 3. Model: BPE (já carregado)
# ------------------------------------------------------------------
print("# Model: BPE (treinado no corpus de Marco Aurélio)")

# ------------------------------------------------------------------
# 4. Post-processing
# ------------------------------------------------------------------
print("# Post-processing (TemplateProcessing — estilo BERT)")
tokenizer.post_processor = TemplateProcessing(
    single="[CLS] $A [SEP]",
    pair="[CLS] $A [SEP] $B:1 [SEP]:1",
    special_tokens=[("[CLS]", 2), ("[SEP]", 3)],
)

# ------------------------------------------------------------------
# Aplicando o pipeline completo
# ------------------------------------------------------------------
frases_encode = [
    "A vida de um homem é o que seus pensamentos fazem dela.",
    "O universo é mudança; a vida é opinião.",
]
for f in frases_encode:
    encoded = tokenizer.encode(f)
    print(f"\nFrase  : {f}")
    print(f"Tokens : {encoded.tokens}")
    print(f"IDs    : {encoded.ids}")

---
## Parte 4 - ByteLevel vs SentencePiece

Comparamos os tokenizadores pré-treinados GPT-2 (ByteLevel) e mT5 (SentencePiece/Unigram) sobre frases do corpus de Marco Aurélio.

In [ ]:
from transformers import AutoTokenizer
import unicodedata

BYTELEVEL_MODEL = "openai-community/gpt2"
SENTPIECE_MODEL = "google/mt5-small"

tok_byte_pt = AutoTokenizer.from_pretrained(BYTELEVEL_MODEL)
tok_spm     = AutoTokenizer.from_pretrained(SENTPIECE_MODEL)

if tok_byte_pt.pad_token is None:
    tok_byte_pt.pad_token = tok_byte_pt.eos_token

# Frase com caracteres acentuados e pontuação — típica do corpus
texto = "A tranqüilidade não é nada além da boa ordenação da mente."
print(f"Texto: {texto}\n")

def encode_details(tokenizer, name):
    enc    = tokenizer(texto, add_special_tokens=True, return_offsets_mapping=True)
    tokens = tokenizer.convert_ids_to_tokens(enc["input_ids"])
    ids    = enc["input_ids"]
    offsets = enc["offset_mapping"]
    print(f"=== {name} ===")
    print("Tokens     :", tokens)
    print("IDs        :", ids)
    print("Qtd tokens :", len(tokens))
    print("Decoded    :", tokenizer.decode(ids))
    print("Offsets    :", offsets)
    print()

encode_details(tok_byte_pt, "ByteLevel (GPT-2)")
encode_details(tok_spm,     "SentencePiece (mT5)")

# Inspecionando os caracteres Unicode da frase
print("Caracteres Unicode:")
for ch in texto:
    name = unicodedata.name(ch, "UNKNOWN")
    print(f"  {repr(ch)} → {name}")

---
## Parte 5 - Avaliação

Métricas de eficiência dos dois modelos treinados (BPE vs WordPiece) sobre uma amostra do corpus de Marco Aurélio, incluindo frases do domínio e frases técnicas fora do domínio.

In [ ]:
tok_bpe = Tokenizer.from_file("tokenizer_bpe.json")
tok_wp  = Tokenizer.from_file("tokenizer_wp.json")

# Corpus de teste: frases dentro e fora do domínio
frases_avaliacao = [
    # Dentro do domínio (espera-se alta eficiência)
    "A felicidade da sua vida depende da qualidade dos seus pensamentos.",
    "A vida de um homem é o que seus pensamentos fazem dela.",
    "A tranquilidade não é nada além da boa ordenação da mente.",
    "O homem conquista o mundo conquistando a si mesmo.",
    "Siga para onde a razão levar.",
    "Olhe para dentro. Dentro está a fonte do bem.",
    # Fora do domínio (espera-se quebra de tokens)
    "SeedMetrics utiliza inteligência artificial e visão computacional.",
    "Anticonstitucionalissimamente é uma palavra longa.",
    "Data science e machine learning transformam o agro.",
    "O sistema de análise de sementes falhou no upload.",
]

def avaliar_modelo(tokenizer, nome, textos):
    dados = []
    for t in textos:
        enc = tokenizer.encode(t)
        dados.append({
            "Modelo":      nome,
            "Texto":       t[:35] + "..." if len(t) > 35 else t,
            "Chars":       len(t),
            "Palavras":    len(t.split()),
            "Tokens":      len(enc.tokens),
            "<UNK>": enc.tokens.count("[UNK]"),
            "Reversível":  tokenizer.decode(enc.ids) == t,
        })
    return pd.DataFrame(dados)

df_bpe = avaliar_modelo(tok_bpe, "BPE",       frases_avaliacao)
df_wp  = avaliar_modelo(tok_wp,  "WordPiece", frases_avaliacao)

df_final = pd.concat([df_bpe, df_wp]).sort_values(["Texto", "Modelo"])
pd.set_option('display.max_colwidth', None)
print("=== TABELA COMPARATIVA ===")
print(df_final.to_string(index=False))

In [ ]:
# --- Métricas agregadas ---
for df, nome in [(df_bpe, "BPE"), (df_wp, "WordPiece")]:
    tpc        = (df["Tokens"] / df["Chars"]).mean()
    tpw        = (df["Tokens"] / df["Palavras"]).mean()
    unk_rate   = (df["<UNK>"].sum() / df["Tokens"].sum()) * 100
    rev_acc    = df["Reversível"].mean() * 100
    print(f"=== {nome} ===")
    print(f"  Tokens por caractere (TPC) : {tpc:.3f}")
    print(f"  Tokens por palavra   (TPW) : {tpw:.3f}")
    print(f"  Percentual de [UNK]        : {unk_rate:.2f}%")
    print(f"  Reversibilidade            : {rev_acc:.1f}%")
    print(f"  Tam. médio de sequência    : {df['Tokens'].mean():.1f} tokens/frase\n")

In [ ]:
# --- Visualização: Quantidade de tokens por frase ---
labels = [t[:25] + "..." if len(t) > 25 else t for t in frases_avaliacao]
x = np.arange(len(labels))
w = 0.35

fig, ax = plt.subplots(figsize=(13, 5))
ax.bar(x - w/2, df_bpe["Tokens"].values, width=w, label="BPE",       color="steelblue")
ax.bar(x + w/2, df_wp["Tokens"].values,  width=w, label="WordPiece", color="tomato", alpha=0.85)
ax.set_xticks(x)
ax.set_xticklabels(labels, rotation=40, ha="right", fontsize=8)
ax.set_ylabel("Quantidade de tokens")
ax.set_title("BPE vs WordPiece — Tokens por frase (corpus Marco Aurélio)")
ax.legend()
ax.axvline(x=5.5, color="gray", linestyle="--", alpha=0.5, label="Domínio | Fora do domínio")
ax.text(2.5, ax.get_ylim()[1]*0.92, "No domínio",    ha="center", color="gray", fontsize=9)
ax.text(8,   ax.get_ylim()[1]*0.92, "Fora do domínio", ha="center", color="gray", fontsize=9)
plt.tight_layout()
plt.show()

In [ ]:
# --- Raio-X do vocabulário: BPE vs WordPiece ---
vocab_bpe = set(tok_bpe.get_vocab().keys())
vocab_wp  = set(tok_wp.get_vocab().keys())

intersecao      = vocab_bpe & vocab_wp
exclusivos_bpe  = vocab_bpe - vocab_wp
exclusivos_wp   = vocab_wp  - vocab_bpe

print("=== Raio-X do Vocabulário ===")
print(f"Total BPE      : {len(vocab_bpe)}")
print(f"Total WP       : {len(vocab_wp)}")
print(f"Em comum       : {len(intersecao)}")
print(f"Exclusivo BPE  : {len(exclusivos_bpe)} tokens")
print(f"Exclusivo WP   : {len(exclusivos_wp)} tokens")

print("\nExemplos exclusivos do BPE (sufixos fundidos):")
print(list(exclusivos_bpe)[:10])

print("\nExemplos exclusivos do WordPiece (prefixos ##):")
print(list(exclusivos_wp)[:10])

---
## Conclusões

| Aspecto | BPE | WordPiece |
|---|---|---|
| **Critério de fusão** | Frequência bruta do par | Maximiza verossimilhança: `freq(AB)/(freq(A)×freq(B))` |
| **Marcação de subpalavras** | Sem marcação especial | Sufixos marcados com `##` |
| **Decoder** | Concatenação direta | Remove os `##` antes de concatenar |
| **Vocabulário** | Tende a fundir sufixos frequentes | Tende a preservar prefixos comuns |
| **Eficiência no domínio** | Alta (corpus português filosófico) | Alta (similar ao BPE neste corpus) |
| **Fora do domínio** | Quebra em caracteres individuais | Idem — ambos não conhecem o vocabulário técnico |

**Observação sobre o corpus:** Por ser um corpus pequeno (100 frases) em português, ambos os modelos apresentam comportamento similar dentro do domínio. Palavras técnicas em inglês ou compostas ("Anticonstitucionalissimamente", "SeedMetrics") geram muitos tokens `[UNK]` ou subpalavras de 1 caractere, evidenciando a importância do corpus de treinamento na qualidade da tokenização.